In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

# PySpark MLlib
<!-- requirement: small_data/gutenberg -->
*Official documentation [here](https://spark.apache.org/docs/latest/mllib-guide.html).*

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
print sc.version

2.0.1


In [3]:
# needed to convert RDDs into DataFrames
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
# Tells Spark to look on the local filesystem
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

## Algorithms

Spark supports a number of machine-learning algorithms.

- Classification and Regression
    - SVM, linear regression
    - SVR, logistic regression
    - Naive Bayes
    - Decision Trees
    - Random Forests and Gradient-Boosted Trees
- Clustering
    - K-means (and streaming K-means)
    - Gaussian Mixture Models
    - Latent Dirichlet Allocation
- Dimensionality Reduction
    - SVD and PCA
- It also has support for lower-level optimization primitives:
    - Stochastic Gradient Descent
    - Low-memory BFGS and L-BFGS

### Parallelized SGD

For linear models like SVM, Linear Regression, and Logistic Regression, the cost function we're trying to optimize is essentially an average over the individual error term from each data point. This is particularly great for parallelization.  For example, in linear regression, recall that the gradient is

$$\begin{align}
\frac{\partial \log(L(\beta))}{\partial \beta} &= \frac{\partial}{\partial \beta} \frac{1}{2}\sum_j \|y_j - X_{j \cdot} \cdot \beta\| \\
&= \frac{1}{2}\sum_j \frac{\partial}{\partial \beta} \|y_j - X_{j \cdot} \cdot \beta\| \\
& = \sum_j y_j - X_{j \cdot} \cdot \beta \\
& \approx \sum_{sample \mbox{ } j} y_j - X_{j \cdot} \cdot \beta
\end{align}$$

The key *mathematical properties* we have used are:

1. the error functions are the sum of error contributions of different training instances
1. linearity of the derivative
1. associativity of addition
1. downsampling giving an unbiased estimator

Since the last sum is over the different training instances and these are stored on different nodes, we can parallelize the computation of the gradient in SGD across multiple nodes.  Of course, we still need to maintain the running weight $\beta$ that has to be present on every node (through a broadcast variable that is updated).  Notice that SVM, Linear Regression, and Logistic Regression all have error functions that are just sums over training instances so SGD can be used for all these algorithms.

Spark's [implementation](http://spark.apache.org/docs/latest/mllib-optimization.html#stochastic-gradient-descent-sgd) uses a tunable minibatch size parameter to sample a percentage of the features RDD. For each iteration, the updated weights are broadcast to the executors, and the update is calculated for each data point and sent back to be aggregated.

Parallelization handles increasing number of sampled data points m quite well since there are no interaction terms and each calculation is independent. Controlling how the algorithm iterates to convergence is also important, and can be done with parameters for the total iterations and step size.

## ML (DataFrame) vs. MLlib (RDD) packages

Confusingly, there are two machine learning APIs in Spark, the `mllib` package based on RDDs and the `ml` package based on DataFrames. For years these have been developed somewhat in parallel, resulting in duplication and asymmetry in functionality.

With Spark 2.0+, `mllib` is in maintenance mode and will likely be deprecated in future in favor of the DataFrame-based API which more closely resembles libraries like Scikit-learn. Below is one example of the RDD-based API; the rest of the notebook will focus on DataFrames.

In [5]:
from pyspark.mllib.regression import LinearRegressionWithSGD, LinearRegressionModel, LabeledPoint
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.linalg import Vector, Vectors
import random

# parameters
TRAINING_ITERATIONS = 10
TRAINING_FRACTION = 0.6

# generate the data
data = sc.parallelize(xrange(1,10001)) \
    .map(lambda x: LabeledPoint(random.random(), [random.random(), random.random(), random.random()]))

# split the training and test sets
splits = data.randomSplit([TRAINING_FRACTION, 1 - TRAINING_FRACTION], seed=42)
training, test = (splits[0].cache(), splits[1])

# train the model
model = LinearRegressionWithSGD.train(training, iterations=TRAINING_ITERATIONS)

# get r2 score
predictions = test.map(lambda x: (float(model.predict(x.features)), x.label))
print RegressionMetrics(predictions).r2

/opt/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


-0.353939672093


Maybe we can improve this by modeling the intercept. Use `<shift-tab>` inside the arguments to bring up the docstring for LinearRegressionWithSGD, and rerun the training with an intercept term.

If you're interested in methods for introspecting some of these objects, the `<tab>` and `<shift-tab>` documentation is good. You can also use `dir` in Python to list all the components of something.

In [6]:
dir(test.take(2)[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'features',
 'label']

## Spark ML
Spark ML implements the ideas of transformers, estimators, and pipelines by standardizing APIS across machine learning algorithms. This can streamline more complex workflows.

The core functionality includes:
* DataFrames - built off Spark SQL, can be created either directly or from RDDs as seen above
* Transformers - algorithms that accept a DataFrame as input and return a DataFrame as output
* Estimators - algorithms that accept a DataFrame as input and return a Transformer as output
* Pipelines - chaining together Transformers and Estimators
* Parameters - common API for specifying hyperparameters

For example, a learning algorithm can be implemented as an Estimator which trains on a DataFrame of features and returns a Transformer which can output predictions based on a test DataFrame.

Full documentation can be found [here](http://spark.apache.org/docs/latest/ml-guide.html)

In [7]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

reviews = [("Prose is well-written, but style is an impediment to learning. Should be called 'Reviewing Spark,' not 'Learning Spark'", 0.0),
            ("Nice Headstart to Spark", 1.0),
            ("Start here: Excellent reference for Spark", 1.0),
            ("Insightful and so Spark-tastic!", 1.0),
            ("Good intro but wordy and lacking details in areas", 0.0),
            ("Best of the Books Currently Available", 1.0),
            ("A good resource for people interested in learning Spark", 1.0),
            ("Great Overview", 1.0)]

test_reviews = [("A decent guided tour of Spark and its major components.", 0.0),
                ("10/10 would buy again", 1.0),
                ("it is simple to follow. well organized. straight ...", 1.0),
                ("Just what you need to get started in Apache Spark.", 1.0),
                ("Very good book for learning Spark", 1.0)]

training = sqlContext.createDataFrame(reviews, ["title", "label"]).cache()
test = sqlContext.createDataFrame(test_reviews, ["title", "label"])

tokenizer = Tokenizer(inputCol="title", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
logreg = LogisticRegression(maxIter=10, regParam=0.01)

tokens = tokenizer.transform(training)
hashes = hashingTF.transform(tokens)
model = logreg.fit(hashes)

# Make predictions on test documents
test_tokens = tokenizer.transform(test)
test_hashes = hashingTF.transform(test_tokens)

prediction = model.transform(test_hashes)
selected = prediction.select("title", "label", "prediction")
for row in selected.collect():
    print(row)

Row(title=u'A decent guided tour of Spark and its major components.', label=0.0, prediction=1.0)
Row(title=u'10/10 would buy again', label=1.0, prediction=1.0)
Row(title=u'it is simple to follow. well organized. straight ...', label=1.0, prediction=1.0)
Row(title=u'Just what you need to get started in Apache Spark.', label=1.0, prediction=1.0)
Row(title=u'Very good book for learning Spark', label=1.0, prediction=1.0)


In [8]:
# Note that if you use a PipelineModel it won't have a coefficients attribute.
model.coefficients

SparseVector(262144, {6368: -0.4672, 9639: 0.4928, 15889: -0.2336, 16332: 0.5375, 22346: 0.7979, 26114: -0.4672, 29952: -1.0939, 39221: 0.4377, 47372: 0.4928, 47822: 0.8852, 48390: 0.7979, 61971: 0.4928, 77044: -0.4672, 81677: -0.4672, 91677: -0.1217, 99585: -0.4672, 103838: 0.4928, 104587: 0.4377, 106119: -0.4672, 113432: -0.356, 114686: 0.4837, 117481: 0.4377, 123535: -1.0939, 136358: -0.4672, 138356: 1.1303, 138912: -1.0939, 139098: -0.4672, 147278: -0.4672, 149001: -0.4672, 149910: 0.4837, 163984: 0.4837, 166027: 0.4928, 167152: -0.4672, 175563: 1.1303, 178534: -0.4672, 188424: 0.8852, 189683: -0.9106, 194536: -0.4672, 205044: 0.1929, 212740: 0.4377, 221048: -1.0939, 222453: -0.356, 227410: 0.4837, 234657: 0.8024, 235528: 0.4837, 238163: 0.4928, 241199: 0.8852, 247066: -1.0939, 261451: -0.4672})

In [9]:
prediction.select(["features", "probability", "prediction"]).show(2)

+--------------------+--------------------+----------+
|            features|         probability|prediction|
+--------------------+--------------------+----------+
|(262144,[9639,160...|[0.00839737493866...|       1.0|
|(262144,[68867,70...|[0.04252139879146...|       1.0|
+--------------------+--------------------+----------+
only showing top 2 rows



**Exercise**: Rewrite the above using a Pipeline.

## Cross-validation and grid search

In [10]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [11]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, logreg])

In [12]:
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(logreg.regParam, [0.1, 0.01]) \
    .build()

In [13]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

*Note*: A more traditional validation set without folding is available in `TrainValidationSplit`.

In [14]:
cvModel = crossval.fit(training)

In [15]:
better_prediction = cvModel.transform(test)

In [16]:
selected = better_prediction.select("title", "label", "prediction")
for row in selected.collect():
    print(row)

Row(title=u'A decent guided tour of Spark and its major components.', label=0.0, prediction=1.0)
Row(title=u'10/10 would buy again', label=1.0, prediction=1.0)
Row(title=u'it is simple to follow. well organized. straight ...', label=1.0, prediction=1.0)
Row(title=u'Just what you need to get started in Apache Spark.', label=1.0, prediction=0.0)
Row(title=u'Very good book for learning Spark', label=1.0, prediction=1.0)


In [17]:
cvModel.bestModel.stages

[Tokenizer_4c0a94a40c20217d2b6b,
 HashingTF_481a81ae25500da5c651,
 LogisticRegression_4351b91117b5942eb244]

In [18]:
cvModel.bestModel.stages[2].coefficients

DenseVector([0.3927, -0.0662, -0.2097, -0.3057, -1.201, -0.2549, 0.1783, -0.9825, -0.54, -1.0403])

In [19]:
training.unpersist()

DataFrame[title: string, label: double]

### Example algorithm: Word2Vec

In [5]:
from pyspark.ml.feature import Word2Vec

# text = sc.parallelize(reviews + test_reviews).map(lambda (line, score): (line.split(" "), score)).toDF(['text', 'score'])
gutenberg = sc.textFile(localpath('small_data/gutenberg/')).map(lambda line: (line.split(" "), 1)).toDF(['text', 'score'])
w2v = Word2Vec(inputCol="text", outputCol="vectors")
model = w2v.fit(gutenberg)
result = model.transform(gutenberg)

In [6]:
vectors = model.getVectors().rdd.map(lambda x: (x.word, x.vector))
print model.findSynonyms('woman', 10).rdd.take(10)

[Row(word=u'man', similarity=0.8720091039411314), Row(word=u'fellow', similarity=0.8715526365110657), Row(word=u'bit', similarity=0.8573996370982878), Row(word=u'girl', similarity=0.8440238269261223), Row(word=u'sweet', similarity=0.8363238716705361), Row(word=u'voice', similarity=0.8332955778996173), Row(word=u'waiter', similarity=0.8332426776296992), Row(word=u'Not', similarity=0.8268376719225548), Row(word=u'woman.', similarity=0.8260626755394919), Row(word=u'charming', similarity=0.8219273180949814)]


In [7]:
king_vec = vectors.lookup('king')[0]
queen_vec = vectors.lookup('queen')[0]
man_vec = vectors.lookup('man')[0]
woman_vec = vectors.lookup('woman')[0]

In [8]:
print king_vec

[-0.0136160766706,-0.029679980129,-0.0187971945852,0.0146245937794,-0.0260057058185,0.0440981797874,0.0411708056927,0.0129473954439,-0.0303609333932,-0.0183322634548,-0.0590302124619,0.00351622304879,-0.0160355623811,-0.0716458708048,0.00539460312575,0.00347200571559,0.000186914927326,-0.00145505182445,0.0385810695589,0.0619655586779,0.0316808745265,-0.00644708890468,-0.0094636855647,0.0398297533393,-0.00672862725332,-0.0308548696339,-0.012208792381,0.0237402208149,0.0482747405767,0.00147257256322,-0.0338950566947,0.0314285010099,-0.032340425998,-0.0320617295802,-0.000477329973364,-0.0184230227023,0.0476760305464,0.0251849628985,-0.0127260303125,0.0447790436447,0.00177274330053,-0.0177723299712,-0.0406013727188,-0.0751792490482,-0.0786015689373,0.0610344968736,-0.0256222747266,0.00955851189792,-0.00771653465927,-0.0190721489489,0.0656533539295,0.00223001209088,0.00118451786693,0.0486329607666,0.0150789804757,0.0243607535958,0.0485647022724,-0.0558753535151,0.00258200173266,0.0128305517

In [9]:
print queen_vec.squared_distance(king_vec)
print queen_vec.squared_distance(woman_vec)
print queen_vec.squared_distance(man_vec)
print queen_vec.squared_distance(king_vec + man_vec - woman_vec)
print queen_vec.squared_distance(king_vec - man_vec + woman_vec)

0.0906354193
0.375758404359
1.26432601996
0.565848466243
0.530651124966


## Feature processing

In [ ]:
vdf = vectors.toDF(["word", "vector"])

In [ ]:
vdf.show(5)

In [ ]:
sample_vector = vdf.select("vector").take(1)[0][0]
print len(sample_vector)

In [ ]:
from pyspark.ml.feature import VectorSlicer

first_slicer = VectorSlicer(inputCol="vector", outputCol="first", indices=[0])
last_slicer = VectorSlicer(inputCol="vector", outputCol="last", indices=[len(sample_vector) - 1])
med_slicer = VectorSlicer(inputCol="vector", outputCol="med", indices=range(45, 55))

In [ ]:
output = med_slicer.transform(last_slicer.transform(first_slicer.transform(vdf)))
output.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["first", "last", "med"],
    outputCol="features")

new_output = assembler.transform(output)
new_output.columns

In [ ]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=0.05, inputCol="features", outputCol="bin_features")
new_output_b = binarizer.transform(new_output)
print new_output_b.select(["features", "bin_features"]).take(1)[0]

### Exercise: Use SVM to predict colon cancer from gene expressions
You can start getting a feel for the MLlib operations by following the [Spark docs example](https://spark.apache.org/docs/1.3.0/mllib-linear-methods.html#linear-support-vector-machines-svms) on this dataset.

#### About the data format: LibSVM
MLlib conveniently provides a data loading method, `MLUtils.loadLibSVMFile()`, for the LibSVM format for which many other languages (R, Matlab, etc.) also have loading methods.  
A dataset of *n* features will have one row per datum, with the label and values of each feature organized as follows:
>{label} 1:{value} 2:{value} ... n:{value}

Take these two datapoints with six features and labels of -1 and 1 respectively as an example:
>-1.000000  1:2.080750 2:1.099070 3:0.927763 4:1.029080 5:-0.130763 6:1.265460  
1.000000  1:1.109460 2:0.786453 3:0.445560 4:-0.146323 5:-0.996316 6:0.555759 

#### About the colon-cancer dataset
This dataset was introduced in the 1999 paper "Broad patterns of gene expression revealed by clustering analysis of tumor and normal colon tissues probed by oligonucleotide arrays." (Available on PNAS)

Here's the abstract of the paper:  
> *Oligonucleotide arrays can provide a broad picture of the state of the cell, by monitoring the expression level of thousands of genes at the same time. It is of interest to develop techniques for extracting useful information from the resulting data sets. Here we report the application of a two-way clustering method for analyzing a data set consisting of the expression patterns of different cell types. Gene expression in 40 tumor and 22 normal colon tissue samples was analyzed with an Affymetrix oligonucleotide array complementary to more than 6,500 human genes. An efficient two-way clustering algorithm was applied to both the genes and the tissues, revealing broad coherent patterns that suggest a high degree of organization underlying gene expression in these tissues. Coregulated families of genes clustered together, as demonstrated for the ribosomal proteins. Clustering also separated cancerous from noncancerous tissue and cell lines from in vivo tissues on the basis of subtle distributed patterns of genes even when expression of individual genes varied only slightly between the tissues. Two-way clustering thus may be of use both in classifying genes into functional groups and in classifying tissues based on gene expression.*

There are 2000 features, 62 data points (40 tumor (label=0), 22 normal (label=1)), and 2 classes (labels) for the colon cancer dataset. 

#### Exit Tickets
1. When would you use `org.apache.spark.mllib.linalg.Vector` versus `breeze.linalg.DenseVector`?
1. Why can SVM, Linear Regression, and Logistic Regression be parallelized?  How would you parallelize KMeans?


In [ ]:
sc.stop()

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*